In [8]:
import os
import shutil
from pathlib import Path
from datetime import datetime
from exiftool import ExifToolHelper
from media_tool.move import get_exif_creation_date_pyexiftool, IMAGE_EXTENSIONS

In [9]:
path = "/mnt/orico/photo/fix-date"

In [10]:
def get_images(path):
    """Organizes image and video files into subfolders based on their creation date."""
    source_path = Path(path)

    if not source_path.exists() or not source_path.is_dir():
        print("Source folder does not exist or is not a directory.")
        return

    # Update to recursively find all media files using rglob
    media_files = []
    for ext in IMAGE_EXTENSIONS:
        media_files.extend(source_path.rglob(f"*{ext}"))
        media_files.extend(source_path.rglob(f"*{ext.upper()}"))

    return media_files

In [17]:
files = get_images(path)
example_file = files[0]
example_file

PosixPath('/mnt/orico/photo/fix-date/2022-03-29/rendered.jpg')

In [38]:
with ExifToolHelper() as et:
    metadata_list = et.get_metadata(example_file)
    metadata = metadata_list[0]  # Get the first (and only) metadata dictionary
metadata

{'SourceFile': '/mnt/orico/photo/fix-date/2022-03-29/rendered.jpg',
 'ExifTool:ExifToolVersion': 13.0,
 'ExifTool:Warning': 'IPTCDigest is not current. XMP may be out of sync',
 'File:FileName': 'rendered.jpg',
 'File:Directory': '/mnt/orico/photo/fix-date/2022-03-29',
 'File:FileSize': 15233646,
 'File:FileModifyDate': '2025:03:17 14:23:43+00:00',
 'File:FileAccessDate': '2025:03:17 14:23:42+00:00',
 'File:FileInodeChangeDate': '2025:03:17 14:23:43+00:00',
 'File:FilePermissions': 100777,
 'File:FileType': 'JPEG',
 'File:FileTypeExtension': 'JPG',
 'File:MIMEType': 'image/jpeg',
 'File:ExifByteOrder': 'MM',
 'File:CurrentIPTCDigest': 'b302739d17541a0b63898106ef7e626d',
 'File:ImageWidth': 4157,
 'File:ImageHeight': 6235,
 'File:EncodingProcess': 0,
 'File:BitsPerSample': 8,
 'File:ColorComponents': 3,
 'File:YCbCrSubSampling': '1 1',
 'JFIF:JFIFVersion': '1 1',
 'JFIF:ResolutionUnit': 0,
 'JFIF:XResolution': 72,
 'JFIF:YResolution': 72,
 'EXIF:Make': 'FUJIFILM',
 'EXIF:Model': 'X-T5',

In [35]:
def create_date_updates(file_path: str, target_year: int) -> dict:
    """
    Creates a dictionary of date field updates for the given file, changing the year
    while preserving all other date components.

    Args:
        file_path: Path to the image file
        target_year: The year to set in the date fields

    Returns:
        Dictionary containing the field updates
    """
    # Date fields we want to modify
    date_fields = [
        "EXIF:ModifyDate",
        "EXIF:DateTimeOriginal",
        "EXIF:CreateDate",
        "XMP:CreateDate",
        "XMP:ModifyDate",
        "XMP:DateCreated",
        "IPTC:DigitalCreationDate",
        "IPTC:DateCreated",
        "Composite:SubSecCreateDate",
        "Composite:SubSecDateTimeOriginal",
        "Composite:SubSecModifyDate",
        "Composite:DateTimeCreated",
        "Composite:DigitalCreationDateTime",
    ]

    updates = {}

    # Read current dates from file
    with ExifToolHelper() as et:
        metadata = et.get_metadata(file_path)
        if metadata:
            metadata = metadata[0]  # Get first (and only) item

            for field in date_fields:
                if field in metadata:
                    current_date_str = str(metadata[field])
                    try:
                        updates[field] = str(target_year) + current_date_str[4:]
                    except ValueError:
                        print(
                            f"Could not parse date for field {field}: {current_date_str}"
                        )
                        continue

    return updates


def apply_date_updates(file_path: str, updates: dict) -> bool:
    """
    Applies the date updates to the specified file using ExifTool.

    Args:
        file_path: Path to the image file
        updates: Dictionary of field updates from create_date_updates()

    Returns:
        Boolean indicating success
    """
    try:
        with ExifToolHelper() as et:
            # Convert updates to ExifTool command format
            params = ["-overwrite_original"]  # Overwrite original file
            for field, value in updates.items():
                params.extend([f"-{field}={value}"])

            # Apply the updates
            et.execute(*params, file_path)
        return True
    except Exception as e:
        print(f"Error updating file {file_path}: {str(e)}")
        return False

In [39]:
for file in files:
    apply_date_updates(file, create_date_updates(file, 2024))